In [1]:
import datetime

import backtrader

import lstm

Using TensorFlow backend.


In [2]:
# datafeed to provide indexes to frames to be used by a strategy
class FrameFeed(backtrader.feed.DataBase):
    params = (
        ("fromdate", datetime.datetime),
        ("todate", datetime.datetime),
        ("frames", None),
    )

    lines = ("datetime", "frame_index", "close")

    def __init__(self):
        super()

    def start(self):
        super(backtrader.feed.DataBase, self).start()
        # TODO: use assignment expressions
        self.frames = (
            frame for frame in enumerate(self.p.frames)
            if self.p.fromdate <= frame[1][-1][-1][0] <= self.p.todate
        )

    def _load(self):
        try:
            i, frame = next(self.frames)
        except StopIteration:
            return False

        self.lines.datetime[0] = backtrader.utils.date2num(frame[-1][-1][0])
        self.lines.frame_index[0] = i

        return True

In [29]:
# strategy to use by backtesting
class TestStrategy(backtrader.Strategy):
    params = (
        ("model", None),
        ("frames", [[[]]]),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt, txt))

    def __init__(self):
        self.closings = self.datas[0].close
        self.frame_index = self.datas[1].frame_index
        self.frames = self.params.frames
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED:')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log("SELL EXECUTED:")

            print(f"Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}")
            print(self.broker.getvalue())

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        current_frame = [[self.closings[i]] for i in range(0, -15, -1)]
        previous_frame = [[self.closings[i]] for i in range(-1, -16, -1)]

        current_normalized_frame = lstm.normalize_frame(current_frame)
        previous_normalized_frame = lstm.normalize_frame(previous_frame)
    
        current_prediction = lstm.predict_sequences_multiple(self.p.model, [current_normalized_frame])
        previous_prediction = lstm.predict_sequences_multiple(self.p.model, [previous_normalized_frame])
        
        current_prediction_denorm = lstm.denormalize_dim(current_prediction[0][0], current_frame[0][0])
        # previous_prediction_denorm = lstm.denormalize_dim(previous_prediction[0][0], previous_frame[0][0])
        
        # print(f"current: {current_frame}, prediction: {current_prediction_denorm}")
        
        pred_diff = current_prediction[0][0] - previous_prediction[0][0]
        actual_diff = current_frame[-1][0] - previous_frame[-1][0]
        # diff = current_prediction_denorm - previous_prediction_denorm
        # diff = current_prediction[0][0] - previous_prediction[0][0]
        
        # print(f"{current_prediction} - {previous_prediction} = {diff}")
        
        # note: track position and buy/sell entire stake
        if self.position and pred_diff < 0 or current_prediction_denorm < current_frame[-1][0]:
            self.log(f"SELL")
            self.order = self.sell()
        else:
            if pred_diff > 0 and current_prediction_denorm > current_frame[-1][0]:
                self.log(f"BUY")
                self.order = self.buy()

In [36]:
class Reverser(backtrader.sizers.FixedSize):

    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data)
#         size = self.p.stake * (1 + (position.size != 0))
#         return size
        if isbuy:
            return cash // data.open[0]
        else:
            return position.size

In [41]:
SYMBOL="AMZN"

# get testing data
times = lstm.get_time_series_daily(SYMBOL, ["4. close"], outputsize="full")
vectors = lstm.times_to_vectors(times, include_time=True)[::-1]

train_vectors, test_vectors = lstm.partition_data(vectors)

train_frames = lstm.get_frames(train_vectors, 15, with_target=True)
test_frames = lstm.get_frames(test_vectors, 15, with_target=False)

train_no_dates = [[[col[1] for col in vector] for vector in frame] for frame in train_frames]
normalized_train = lstm.normalize_frames(train_no_dates)
x_train, y_train = lstm.seperate_xy(normalized_train)

In [42]:
# setup model
model = lstm.setup_lstm_model(x_train, y_train)

compilation time :  0.018516063690185547
Train on 4032 samples, validate on 213 samples
Epoch 1/1
4032/4032 [==============================] - 2s 606us/step - loss: 15.8811 - val_loss: 16.5643


In [43]:
# setup inital testing strategy
cerebro = backtrader.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.addsizer(Reverser)
cerebro.broker.setcommission(commission=0.001)

# add data and model to strategy
cerebro.addstrategy(TestStrategy, model=model, frames=test_frames)

from_date = datetime.datetime(2018, 1, 1)
to_date = datetime.datetime(2019, 1, 1)

cerebro.adddata(
    backtrader.feeds.YahooFinanceData(
        dataname=SYMBOL,
        fromdate=from_date,
        todate=to_date
    )
)

cerebro.adddata(
    FrameFeed(
        frames=test_frames,
        fromdate=from_date,
        todate=to_date
    )
)

print(f'Beginning Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Beginning Portfolio Value: 100000.0
2018-01-02, BUY
2018-01-02, Order Canceled/Margin/Rejected
2018-01-02, BUY
2018-01-03, Order Canceled/Margin/Rejected
2018-01-17, BUY
2018-01-18, BUY EXECUTED:
Price: 1293.95, Cost: 98340.2, Comm: 98.3402
99853.77979999999
2018-01-18, SELL
2018-01-19, SELL EXECUTED:
Price: 1312.0, Cost: 98340.2, Comm: 99.712
101173.7478
2018-01-24, BUY
2018-01-25, BUY EXECUTED:
Price: 1368.0, Cost: 99864.0, Comm: 99.86399999999999
101800.2338
2018-01-25, SELL
2018-01-26, SELL EXECUTED:
Price: 1392.01, Cost: 99864.0, Comm: 101.61672999999999
102724.99707
2018-01-31, BUY
2018-02-01, BUY EXECUTED:
Price: 1445.0, Cost: 101150.0, Comm: 101.15
98773.84707
2018-02-01, BUY
2018-02-02, Order Canceled/Margin/Rejected
2018-02-02, SELL
2018-02-05, SELL EXECUTED:
Price: 1402.62, Cost: 101150.0, Comm: 98.1834
99559.06367
2018-02-05, BUY
2018-02-06, BUY EXECUTED:
Price: 1361.46, Cost: 95302.2, Comm: 95.30220000000001
105160.36146999999
2018-02-06, SELL
2018-02-07, SELL EXECUTED:
Pr

2018-08-10, BUY
2018-08-13, BUY EXECUTED:
Price: 1898.5, Cost: 115808.5, Comm: 115.8085
115740.95231999997
2018-08-13, SELL
2018-08-14, SELL EXECUTED:
Price: 1919.39, Cost: 115808.5, Comm: 117.08279
117038.45952999998
2018-08-15, BUY
2018-08-16, BUY EXECUTED:
Price: 1903.94, Cost: 116140.34, Comm: 116.14034
115859.69918999998
2018-08-16, BUY
2018-08-16, BUY
2018-08-17, BUY
2018-08-17, BUY
2018-08-20, BUY
2018-08-20, BUY
2018-08-21, SELL
2018-08-22, SELL EXECUTED:
Price: 1876.64, Cost: 116140.34, Comm: 114.47504
115142.54414999997
2018-08-23, BUY
2018-08-24, BUY EXECUTED:
Price: 1910.51, Cost: 114630.6, Comm: 114.6306
114720.71354999997
2018-08-24, BUY
2018-08-24, BUY
2018-08-27, SELL
2018-08-28, SELL EXECUTED:
Price: 1937.73, Cost: 114630.6, Comm: 116.2638
116544.84974999996
2018-08-30, BUY
2018-08-31, BUY EXECUTED:
Price: 2007.0, Cost: 116406.0, Comm: 116.406
116759.62374999997
2018-08-31, SELL
2018-09-04, SELL EXECUTED:
Price: 2026.5, Cost: 116406.0, Comm: 117.537
117441.90674999997
